In [288]:
!pip install selenium
!pip install bs4

In [289]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.by import By

In [290]:
main_link = 'https://www.tokopedia.com/search?st=&q=handphone&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource='

path = 'data\chrome.exe'

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.binary_location = path  # Set Chrome binary location

driver = webdriver.Chrome(options=chrome_options)
time.sleep(10)
driver.get(main_link)

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4916\1561910418.py:3: SyntaxWarning: invalid escape sequence '\c'
  path = 'data\chrome.exe'


In [291]:
html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")

In [292]:
shop_name, product_name, product_price, sold = [], [], [], []

def wait_for_elements(class_name, timeout=10):
    try:
        elements = WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, class_name))
        )
        return elements
    except:
        return []

def scrape_page():
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    product_elements = soup.find_all('div', class_='css-1nylpq2')
    if not product_elements:
        print("Nama produk tidak ditemukan.")
    else:
        for i in product_elements:
            product_name.append(i.text.strip())
            print(f"Nama Produk: {i.text.strip()}")
            
    price_elements = soup.find_all('div', class_='price')
    if not price_elements:
        print("Harga produk tidak ditemukan.")
    else:
        for i in price_elements:
            product_price.append(i.text.strip())
            print(f"Harga Produk: {i.text.strip()}")

    shop_elements = soup.find_all('h2', class_='css-1wdzqxj-unf-heading e1qvo2ff2')
    if not shop_elements:
        print("Nama toko tidak ditemukan.")
    else:
        for i in shop_elements:
            shop_name.append(i.text.strip())  
            print(f"Nama Toko: {i.text.strip()}")
    
    sold_elements = soup.find_all('div', class_='items')
    if not sold_elements:
        print("tidak ditemukan.")
    else:
        for i in sold_elements:
            sold.append(i.text.strip())
            print(f"Terjual: {i.text.strip()}")

time.sleep(10)
menu_elements = wait_for_elements('css-5wh65g')

if not menu_elements:
    print("Menu tidak ditemukan, keluar")
else:
    total_elements = len(menu_elements)
    max_menu_elements = 2  
    print(f"Total menu ditemukan: {total_elements}, akan memproses maksimal {max_menu_elements} menu")

    for attempt in range(3):
        for index in range(min(total_elements, max_menu_elements)):
            try:
                
                menu_elements = wait_for_elements('css-5wh65g')
                
                print(f"Mengklik menu {index+1} dari maksimal {max_menu_elements} menu")
                menu_elements[index].click()

                time.sleep(10)
                scrape_page()
                time.sleep(10)

                driver.back()
                time.sleep(10)

                menu_elements = wait_for_elements('css-5wh65g')

            except Exception as e:
                print(f"Error saat mengklik elemen {index+1}: {e}")
                break  
        else:
            break

driver.quit()

Total menu ditemukan: 85, akan memproses maksimal 2 menu
Mengklik menu 1 dari maksimal 2 menu
Nama Produk: Samsung Galaxy S24 FE [8/256] - [Blue, Graphite, Gray] Smartphone AI, AI Phone, Handphone AI, Kamera 50MP, Layar besar, RAM besar, Baterai tahan lama - Blue
Harga Produk: Rp9.999.000
Nama Toko: Samsung Official Store
Terjual: Terjual 40+•Diskusi (4)
Mengklik menu 2 dari maksimal 2 menu
Nama Produk: Infinix Smart 8 Pro 8/128GB- Up to 16GB Extended RAM - 6.6" - Timber Black
Harga Produk: Rp1.369.000
Nama Toko: Infinix Official Store
Terjual: Terjual 3 rb+•bintang 4.9 (1.367 rating)•Diskusi (39)


In [293]:
min_length = min(len(product_name), len(product_price), len(shop_name), len(sold))

shop_name = shop_name[:min_length]
product_name = product_name[:min_length]
product_price = product_price[:min_length]
sold = sold[:min_length]

data = {
    'Nama Toko': shop_name,
    'Nama Produk': product_name,
    'Harga Produk': product_price,
    'sold': sold
}
df = pd.DataFrame(data)
df.to_csv('data/hasil_scraping.csv', index=False)

In [294]:
import pandas as pd
data = pd.read_csv("data/hasil_scraping.csv")
data.head()

,Nama Toko,Nama Produk,Harga Produk,sold
0,Samsung Official Store,"Samsung Galaxy S24 FE [8/256] - [Blue, Graphit...",Rp9.999.000,Terjual 40+•Diskusi (4)
1,Infinix Official Store,Infinix Smart 8 Pro 8/128GB- Up to 16GB Extend...,Rp1.369.000,Terjual 3 rb+•bintang 4.9 (1.367 rating)•Disku...
